In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-classification-finetuning/sample_submission.csv
/kaggle/input/llm-classification-finetuning/train.csv
/kaggle/input/llm-classification-finetuning/test.csv


In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

In [4]:
# Load data
train = pd.read_csv("/kaggle/input/llm-classification-finetuning/train.csv")
test = pd.read_csv("/kaggle/input/llm-classification-finetuning/test.csv")

In [5]:
train.head(2)

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0


In [6]:
test.head(2)

,id,prompt,response_a,response_b
0,136060,"[""I have three oranges today, I ate an orange ...","[""You have two oranges today.""]","[""You still have three oranges. Eating an oran..."
1,211333,"[""You are a mediator in a heated political deb...","[""Thank you for sharing the details of the sit...","[""Mr Reddy and Ms Blue both have valid points ..."


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57477 entries, 0 to 57476
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              57477 non-null  int64 
 1   model_a         57477 non-null  object
 2   model_b         57477 non-null  object
 3   prompt          57477 non-null  object
 4   response_a      57477 non-null  object
 5   response_b      57477 non-null  object
 6   winner_model_a  57477 non-null  int64 
 7   winner_model_b  57477 non-null  int64 
 8   winner_tie      57477 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 3.9+ MB


In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          3 non-null      int64 
 1   prompt      3 non-null      object
 2   response_a  3 non-null      object
 3   response_b  3 non-null      object
dtypes: int64(1), object(3)
memory usage: 228.0+ bytes


In [9]:
#EDA
print("Mean winner_model_a:", train["winner_model_a"].mean())
print("Mean winner_model_b:", train["winner_model_b"].mean())
print("Mean winner_tie:", train["winner_tie"].mean())


Mean winner_model_a: 0.34907876193955845
Mean winner_model_b: 0.341910677314404
Mean winner_tie: 0.30901056074603755


In [10]:
print("Std winner_model_a:", train["winner_model_a"].std())
print("Std winner_model_b:", train["winner_model_b"].std())
print("Std winner_tie:", train["winner_tie"].std())

Std winner_model_a: 0.47668305324710053
Std winner_model_b: 0.4743539615785192
Std winner_tie: 0.46208954661193535


In [17]:
# Count model_a
win_a = train.groupby("model_a")["winner_model_a"].mean()

# Count model_b
win_b = train.groupby("model_b")["winner_model_b"].mean()

# Concat
model_winrate = pd.concat([win_a, win_b], axis=1).fillna(0)
model_winrate.columns = ["win_as_A", "win_as_B"]

# total winrate
model_winrate["total_winrate"] = (model_winrate["win_as_A"] + model_winrate["win_as_B"]) / 2
model_winrate.sort_values("total_winrate", ascending=False)

,win_as_A,win_as_B,total_winrate
gpt-4-1106-preview,0.548940,0.553788,0.551364
gpt-3.5-turbo-0314,0.568111,0.524390,0.546251
gpt-4-0125-preview,0.525573,0.502530,0.514051
gpt-4-0314,0.494969,0.471744,0.483357
claude-1,0.443478,0.434998,0.439238
...,...,...,...
stablelm-tuned-alpha-7b,0.175127,0.167109,0.171118
llama-13b,0.154676,0.167286,0.160981
chatglm3-6b,0.166998,0.150206,0.158602
dolly-v2-12b,0.161209,0.148883,0.155046


In [18]:
# Preprocess text
train["text"] = train["prompt"].astype(str) + " " + train["response_a"].astype(str) + " " + train["response_b"].astype(str)
test["text"]  = test["prompt"].astype(str)  + " " + test["response_a"].astype(str)  + " " + test["response_b"].astype(str)

In [19]:
# Targets
y = np.argmax(train[["winner_model_a","winner_model_b","winner_tie"]].values, axis=1)

In [20]:
# Vectorizer
vectorizer = TfidfVectorizer(
    max_features=5000,  
    ngram_range=(1,2),  
    stop_words="english"
)

In [21]:
# Models
log_reg = LogisticRegression(max_iter=2000, class_weight="balanced", C=2.0)
rf = RandomForestClassifier(n_estimators=300, max_depth=20, n_jobs=-1, random_state=42)

clf = VotingClassifier(
    estimators=[("lr", log_reg), ("rf", rf)],
    voting="soft"
)

In [22]:
# Train
X = vectorizer.fit_transform(train["text"])
clf.fit(X, y)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=2.0, class_weight='balanced',
                                                 max_iter=2000)),
                             ('rf',
                              RandomForestClassifier(max_depth=20,
                                                     n_estimators=300,
                                                     n_jobs=-1,
                                                     random_state=42))],
                 voting='soft')

In [23]:
# Predict on test
X_test = vectorizer.transform(test["text"])
probs = clf.predict_proba(X_test)

In [24]:
# Submission
submission = pd.DataFrame({
    "id": test["id"],
    "winner_model_a": probs[:,0],
    "winner_model_b": probs[:,1],
    "winner_tie": probs[:,2],
})

In [25]:
from IPython.display import display
display(submission.head())

,id,winner_model_a,winner_model_b,winner_tie
0,136060,0.243623,0.326923,0.429454
1,211333,0.318474,0.332797,0.348728
2,1233961,0.343822,0.420200,0.235978


In [26]:
submission.to_csv("submission.csv", index=False)
print("Submission file created successfully: submission.csv")

Submission file created successfully: submission.csv
